In [1]:
import os
import pandas as pd
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from scipy import sparse, io
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Twitter

# import pyspark 
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Pi") 

In [2]:
# RDD 생성
train = sc.textFile("/user/data/Test_Train_set/train.csv")

In [4]:
# .csv에서 리뷰,라벨만 가져온 RDD 생성(필요x)
x_train = list([x for x in train.map(lambda x:x[:-3]).toLocalIterator()])
y_train = list([x for x in train.map(lambda x:x[-1:]).toLocalIterator()])

In [5]:
# 형태소분석 함수
twit = Twitter()
def tokenizer(text):
    return twit.morphs(text)

In [6]:
# train.csv에서 리뷰만 가져와서 벡터화
vect_1_3 = TfidfVectorizer(ngram_range=(1,5), min_df = 3, tokenizer=tokenizer).fit(x_train)

In [7]:
nx_train_1_3 = vect_1_3.transform(x_train)

In [ ]:
# SVM 최적옵션 찾기
def svc_param_selection(X, y, nfolds):
    Cs = [0.01, 0.1, 1, 10]
    gammas = [0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

print(svc_param_selection(nx_train_1_3, y_train, 5))

In [ ]:
# 최적옵션으로 SVM 정의
clf = svm.SVC(kernel='linear', C = 1.0)
clf.fit(nx_train_1_3, y_train)